# 10위권 음원의 전주와 이번주 순위 평균 비교

## 라이브러리

In [9]:
import pandas as pd
import os

In [15]:
# f_path = 'c:/data/Wantreez/Crawling/music'
# try:
#     os.chdir(f_path)
# except:
#     pass

In [16]:
code_path = os.getcwd().replace('\\', '/')
code_path

'c:/data/Wantreez/Crawling/music'

In [17]:
# 전 주 파일 날짜(일주일)
start_date = '2022-04-25'
end_date = '2022-05-01'

In [18]:
# 비교할 파일 날짜(일주일)
new_start_date = '2022-05-02'
new_end_date = '2022-05-08'
new_dates_df = pd.date_range(new_start_date, new_end_date).astype(str)

In [19]:
# 음원 사이트
site_list = ['flo', 'bugs', 'genie', 'melon', 'vibe']
top10_df = pd.DataFrame()
for site_name in site_list:
    folder_path = f'/crawled_data/mod_{site_name}/'
    path = code_path + folder_path
    try:
        os.chdir(path)
    except:
        pass

    # 파일 이름들
    file_list = os.listdir()

    # 전체 df 생성
    total_df = pd.DataFrame()
    for one in file_list:
        temp = pd.read_excel(one)
        total_df = pd.concat([total_df, temp], ignore_index=True)
    total_df['날짜'] = total_df['날짜'].astype(str)

    # 전주, 이번주 df
    before_df = total_df[ (total_df['날짜'] >= start_date) & (total_df['날짜'] <= end_date)].reset_index(drop=True)
    new_df = total_df[ (total_df['날짜'] >= new_start_date) & (total_df['날짜'] <= new_end_date)].reset_index(drop=True)

    # 전주, 이번주 Top10 df
    before_top10_df = before_df[before_df['순위'] <= 10].reset_index(drop=True)
    current_top10_df = new_df[new_df['순위'] <= 10].reset_index(drop=True)
    # 전주 Top10 음원들의 순위 평균
    mean_before_top10_df = before_top10_df.groupby(['사이트', '곡', '가수']).agg('mean').reset_index()
    mean_before_top10_df['순위'] = round(mean_before_top10_df['순위'], 1)
    mean_before_top10_df.columns = ['사이트', '곡', '가수', '과거순위평균']

    # 이번주 Top10 음원들의 순위 평균
    mean_current_top10_df = current_top10_df.groupby(['사이트', '곡', '가수']).agg('mean').reset_index()
    mean_current_top10_df['순위'] = round(mean_current_top10_df['순위'], 1)
    mean_current_top10_df.columns = ['사이트', '곡', '가수', '현재순위평균']

    # 전주, 이번주 Top10 df 병합
    bvsc_df = pd.merge(mean_before_top10_df, mean_current_top10_df, on=['곡', '가수'], how='inner')
    bvsc_df = bvsc_df.drop('사이트_y', axis=1)
    bvsc_df = bvsc_df.rename(columns={'사이트_x':'사이트'})
    
    # 전주 대비 상승폭
    bvsc_df['전주 대비 상승폭'] = bvsc_df['현재순위평균'] - bvsc_df['과거순위평균']
    bvsc_df = bvsc_df[['현재순위평균', '곡', '가수', '전주 대비 상승폭', '사이트', '과거순위평균']]
    top10_df = pd.concat([top10_df, bvsc_df])
    
# 파일 저장
save_folder = f'/crawled_data/outlier_data/{new_start_date}_{new_end_date}/'
path = code_path + save_folder
file_name = f'{new_start_date}_{new_end_date}_Top10_rank_change_df.xlsx'
top10_df.to_excel(path + file_name, index=False, encoding='utf-8')
print(f'{file_name} 출력완료.')

2022-05-02_2022-05-08_Top10_rank_change_df.xlsx 출력완료.


# 코드 마지막